<a href="https://colab.research.google.com/github/jiiwon129/ESAA/blob/main/ESAA_YB_WEEK12_Dacon_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **제주 특산물 가격 예측 AI 경진대회**

https://dacon.io/competitions/official/236176/data

참고 수상작:

https://dacon.io/competitions/official/236176/codeshare/9381?page=1&dtype=recent

#### **주제 설명**

제주도 특산물의 가격을 예측하는 AI 모델 개발 및 인사이트 발굴
- 제주도의 대표적인 특산물의 가격을 예측하는 AI 모델 개발

#### **데이터 설명**

1. train.csv
- 2019년 01월 01일부터 2023년 03월 03일까지의 유통된 품목의 가격 데이터
  - item: 품목 코드
  - TG : 감귤
  - BC : 브로콜리
  - RD : 무
  - CR : 당근
  - CB : 양배추
  - corporation : 유통 법인 코드
    - 법인 A부터 F 존재
  - location : 지역 코드
  - J : 제주도 제주시
  - S : 제주도 서귀포시
  - supply(kg) : 유통된 물량, kg 단위
  - price(원/kg) : 유통된 품목들의 kg 마다의 가격, 원 단위

2. international_trade.csv
- 관련 품목 수출입 정보
- 중량 단위 kg
- 금액 단위 천 달러

3. test.csv
- 2023년 03월 04일부터 2023년 03월 31일까지의 데이터

#### **EDA & Feature Engineering**

1. 날짜 정보
- 날짜 데이터 이해를 위해 'timestamp' 분해

  -> 년/월/일/요일/주 파생변수 생성

- 연도별 가격 평균을 통해 가격 상승 파악

  -> year_month(누적 월: 1개월-50개월), week_num(누적 주차: 1주-250주) 시계열 파생변수 생성

- 요일별 가격 평균을 통해 일요일 특산물 값이 0

  -> 공휴일 거래가 이뤄지지 않을 것이라 판단 후 공휴일 파생변수 생성

2. 품목별 정보
- 'TG'와 'TG 외 품목' 특성 차이가 나고 'TG' 품목의 Price 특성이 두드러짐

  -> 'TG'와 'TG 외 품목' 프로세스를 다르게 설정

  -> Price에 루트를 씌워 예측 편차 감소

  -> 0값과 0이 아닌 값을 확실히 하기 위해 공휴일이지만 쉬지 않는 날의 공휴일 변수 수정

#### **Validation Strategy**

- 특정 달에 맞춰진 학습을 방지하기 위해 Cross Validation 평가

  -> 시간의 순서가 보존된 TSCV(Time Series Cross Validation) 수행

- Valid Set을 Test set과 동일한 28일로 고정시키고, Train set를 점진적으로 늘려가며 Cross Validation 수행

#### **Modeling**

1. TG 외 품목 Model
  
  1) Feature Engineering: 이상치 제거
  
  2) Ensemble: Catboost, XGBoost

  3) Time Series Cross Validation

  4) 최종 가격 예측

2. TG Model

  - TG1: Catboost + XGBoost의 앙상블
    - [ 장점 ] 2019~2022년 3월의 종합적인 패턴을 가장 잘 반영

  - TG2: Catboost 단일 모델
    - [ 장점 ] 리더보드 평가에서 가장 좋은 성능을 보임

  1) Feature Engineering: 공휴일 수정, Price값 루트 씌우기

  2) TG1와 TG2 Time Series Cross Validation 결과의 평균

  3) 최종 가격 예측

$+$ 0값 처리
- 품목별로 0값을 제외한 Price의 최솟값을 확인하고, 최솟값보다 작은 값을 0으로 처리
  - 모델이 제대로 예측 못한 0값을 확실히하기 위함

#### **차별점 및 배울점**

- 시간 변수에서 지금까지의 분석에서 년, 월까지만 분해해서 진행했었는데 더 정확한 예측을 위해 년, 월뿐만 아니라 일, 요일, 주까지 분해하여 변수 생성함
  - 분해뿐만 아니라 분해 후 시간의 흐름에 따른 변화 파악을 위해 누적 시간 정보 파생변수를 생성한 점

-> 시계열 정보를 자세하게 전처리하는 방법을 배울 수 있었음

- 품목별로 데이터 분포를 확인해보고, 특정 품목과 다른 품목의 두드러지는 차이를 통해 이를 기준으로 모델링을 진행한 점

- TG 모델에서 단일 모델링과 앙상블 모델링을 진행하고 후에 평균으로 최종 예측을 진행한 점
  - 각 모델의 장점을 합칠 수 있는 방법을 확인

- 이상치 제거도 전처리에서 매우 중요한 부분임을 깨달음